In [1]:
from FoodRatingDataSet import FoodRatingDataset

SEQ_LEN = 5

data_set = FoodRatingDataset()

#Test
from Final_Project_Test.TestDataSet import TestDataset
data_set = TestDataset()

NUM_ING = data_set.NUM_ING
OTHER_FEATURES = data_set.OTHER_FEATURES
NUM_RECIPES = data_set.NUM_RECIPES

(<tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 2, 3, 4, 5]])>, <tf.Tensor: shape=(1, 5, 20), dtype=float32, numpy=
array([[[1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
         1., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
         0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1., 0.]]], dtype=float32)>, <tf.Tensor: shape=(1, 5, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 5,  8,  9],
        [ 6,  2, 10],
        [ 5,  8,  1],
        [ 7,  6,  9]]])>)


In [2]:
from FoodRecommenderModelSequence import FoodRecommenderModelSequence
import tensorflow as tf
import keras

ids, ing, ofe = tf.keras.Input(shape=[SEQ_LEN]), tf.keras.Input(shape=[SEQ_LEN, NUM_ING]), tf.keras.Input(
    shape=[SEQ_LEN, OTHER_FEATURES])

x = FoodRecommenderModelSequence(recipe_count=NUM_RECIPES, seq_len=SEQ_LEN)((ids, ing, ofe))

mod = keras.Model((ids, ing, ofe), x)
mod.summary()

x_id_pre: Tensor("food_recommender_model_sequence/recipe_embedding/embedding/embedding_lookup/Identity_1:0", shape=(None, 5, 32), dtype=float32)
POS_emb: Tensor("food_recommender_model_sequence/recipe_embedding/embedding_1/embedding_lookup/Identity_1:0", shape=(5, 32), dtype=float32)
x_id: Tensor("food_recommender_model_sequence/recipe_embedding/add/add:0", shape=(5, 5, 32), dtype=float32)
x_ing: Tensor("food_recommender_model_sequence/recipe_embedding/dense/BiasAdd:0", shape=(None, 5, 128), dtype=float32)
x_o: Tensor("food_recommender_model_sequence/recipe_embedding/dense_1/BiasAdd:0", shape=(None, 5, 16), dtype=float32)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5)]          0           []                               
                                                          

In [3]:
%load_ext tensorboard

In [4]:
from tensorflow.keras.callbacks import TensorBoard
import time

data = data_set.data()

mod = FoodRecommenderModelSequence(recipe_count=NUM_RECIPES, seq_len=SEQ_LEN)
mod.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])

tensorboard = TensorBoard(log_dir=f"logs/recommend_{int(time.time())}")
#mod.fit(data, label, epochs=10, validation_split=0.3, callbacks=[tensorboard])
mod.fit(data, epochs=10, callbacks=[tensorboard])

Epoch 1/10
DATA:
X: (<tf.Tensor 'IteratorGetNext:0' shape=(None, 5) dtype=int32>, <tf.Tensor 'IteratorGetNext:1' shape=(None, 5, 20) dtype=float32>, <tf.Tensor 'IteratorGetNext:2' shape=(None, 5, 3) dtype=int32>)
Y: Tensor("IteratorGetNext:3", shape=(None,), dtype=int32)
x_id_pre: Tensor("food_recommender_model_sequence_1/recipe_embedding_1/embedding_2/embedding_lookup/Identity_1:0", shape=(None, 5, 32), dtype=float32)
POS_emb: Tensor("food_recommender_model_sequence_1/recipe_embedding_1/embedding_3/embedding_lookup/Identity_1:0", shape=(5, 32), dtype=float32)
x_id: Tensor("food_recommender_model_sequence_1/recipe_embedding_1/add_3/add:0", shape=(5, 5, 32), dtype=float32)
x_ing: Tensor("food_recommender_model_sequence_1/recipe_embedding_1/dense_6/BiasAdd:0", shape=(None, 5, 128), dtype=float32)
x_o: Tensor("food_recommender_model_sequence_1/recipe_embedding_1/dense_7/BiasAdd:0", shape=(None, 5, 16), dtype=float32)
DATA:
X: (<tf.Tensor 'IteratorGetNext:0' shape=(None, 5) dtype=int32>, <

InvalidArgumentError:  ConcatOp : Dimensions of inputs should match: shape[0] = [5,5,32] vs. shape[1] = [1,5,128]
	 [[node food_recommender_model_sequence_1/recipe_embedding_1/concatenate_1/concat
 (defined at C:\Users\Arne\PycharmProjects\iannwtfael\venv\lib\site-packages\keras\backend.py:3224)
]] [Op:__inference_train_function_2655]

Errors may have originated from an input operation.
Input Source operations connected to node food_recommender_model_sequence_1/recipe_embedding_1/concatenate_1/concat:
In[0] food_recommender_model_sequence_1/recipe_embedding_1/add_3/add (defined at C:\Users\Arne\PycharmProjects\iannwtfael\venv\lib\site-packages\keras\layers\merge.py:256)	
In[1] food_recommender_model_sequence_1/recipe_embedding_1/dense_6/BiasAdd (defined at C:\Users\Arne\PycharmProjects\iannwtfael\venv\lib\site-packages\keras\layers\core\dense.py:210)	
In[2] food_recommender_model_sequence_1/recipe_embedding_1/dense_7/BiasAdd:	
In[3] food_recommender_model_sequence_1/recipe_embedding_1/

In [ ]:
%tensorboard --logdir logs

In [ ]:
from FoodRecommenderModel import FoodRecommenderModel
import tensorflow as tf
import keras

ids, ing, ofe = tf.keras.Input(shape=[SEQ_LEN]), tf.keras.Input(shape=[SEQ_LEN, NUM_ING]), tf.keras.Input(
    shape=[SEQ_LEN, OTHER_FEATURES])
tar_id, tar_ing, tar_ofe = tf.keras.Input(shape=[1]), tf.keras.Input(shape=[1, NUM_ING]), tf.keras.Input(
    shape=[1, OTHER_FEATURES])

x = FoodRecommenderModel()(((ids, ing, ofe), (tar_id, tar_ing, tar_ofe)))

mod = keras.Model((ids, ing, ofe, tar_id, tar_ing, tar_ofe), x)
mod.summary()


In [ ]:
from FoodRecommenderModelSelfAttention import FoodRecommenderModelSelfAttention
import tensorflow as tf
import keras

ids, ing, ofe = tf.keras.Input(shape=[SEQ_LEN]), tf.keras.Input(shape=[SEQ_LEN, NUM_ING]), tf.keras.Input(
    shape=[SEQ_LEN, OTHER_FEATURES])
tar_id, tar_ing, tar_ofe = tf.keras.Input(shape=[1]), tf.keras.Input(shape=[1, NUM_ING]), tf.keras.Input(
    shape=[1, OTHER_FEATURES])

x = FoodRecommenderModelSelfAttention()(((ids, ing, ofe), (tar_id, tar_ing, tar_ofe)))

mod = keras.Model((ids, ing, ofe, tar_id, tar_ing, tar_ofe), x)
mod.summary()
